The purpose of this project is to use the Ames Housing Dataset to preform predictive analysis on the sales price of homes. The dataset can be found at https://www.kaggle.com/datasets/prevek18/ames-housing-dataset. This dataset contains a number of attributes for homes such as square footage, overall quality as rated by real estate agents, and others that were sold in Ames, Iowa in the early 2000s. This dataset contains the final price the home was sold at, allowing for pricing predictions

To begin, we need to know the scope of the data we are working with. Luckily, a column definitions document can be found at https://jse.amstat.org/v19n3/decock/DataDocumentation.txt. Using this documentation, we can see that there are 2930 total observations within the dataset, along with 82 total features. Before we can start predictive analysis, it is clear that dimensionality reduction is required to reduce the chances of over-fitting. There are number of routes that could be taken to reduce the number of features, but it is often best to start with the simplest. We will naively select top_top_columns based on their correlation to the dependent variable, the sales price.

In [130]:
import matplotlib.pyplot as plt

import seaborn as sn
import matplotlib.pyplot as plt

In [131]:
import pandas as pd

df = pd.read_csv("AmesHousing.csv")

In [178]:
from math import ceil
from sklearn import preprocessing
import plotly.express as px

# Converts non-numeric columns into categorical numerics so correlation can be determined
binned_df = df.apply(preprocessing.LabelEncoder().fit_transform)

# Creates a correlation matrix
top_columns = binned_df.corr('pearson').abs()

# Sorts the correlations based on their correlation to the sales price
top_columns = top_columns.sort_values('SalePrice', ascending=False)

# Captures the 20%, or 16, highest correlated columns, then stores the names of those columns as a list
top_columns = top_columns.head(ceil(len(df.columns) * .2))
top_columns = top_columns.index.tolist()

# Creates the heatmap to display the correlation
fig = px.imshow(df[top_columns].corr('pearson'), text_auto=True, aspect="auto")
fig.show()

Here we can see some of the more impactful features of the dataset. I chose to use a person correlation because our dataset contains several continuous variables, including the dependent, sales price. However, not every column combination meets this requirement, but person still proves to be a decent choice for determining correlation.

If we observe the far left column, we can see the correlation between our top 20% columns and the sales price. Our best preforming column is "Overall Qual", which is a 1 to 10 ranking of the overall quality of the home. This makes sense logically as the quality of the home would obviously have a large impact on the price of a home. The next three columns all refer to the size of the garage, all of which have a positive correlation, implying that prospective home buyers prefer larger homes over smaller. The remaining attributes also show positive correlations, focusing on overall home size and amenities such as fireplaces and time since last remodel/addition.

This is a simplistic approach for determining key features for use in machine learning, however there is a lot more that could be done to improve the quality of our feature engineering. Several columns in this dataset are categorical, meaning there might be specific values that have high correlation, but are being hidden by the overall correlation of the column. To help identify impactful "sub features", we will preform one-hot encoding to show the correlation between categorical features.

In [133]:
import numpy as np
from sklearn.preprocessing import OneHotEncoder

top_features = df[top_columns]
categorical_columns = ['Garage Cars', 'Full Bath', 'Fireplaces', 'TotRoms AbvGrd', 'Overall Qual']

one_hot_encoder = OneHotEncoder(sparse=False, drop='last')


In [183]:
split_columns_df = pd.DataFrame()

# Loops through each column to create a one hot
for x in range(1, len(top_columns)):
    
    # If the column is continuous, check the correlation value and determine 
    if 1. * df[top_columns[x]].nunique() / df[top_columns[x]].count() >= 0.01:
        
        # Creates the correlation for the column, then captures the relation to SalePrice and stores it
        corr = df[top_columns].corr()['SalePrice']
        split_columns_df = (df[top_columns[x]]).to_frame().merge(temp_df, how='outer', left_index=True, right_index=True)
        continue

    # Since the column is continuous, create the dummy variables, then find the correlation between SalePrice and dummies
    onehot_df = pd.get_dummies(df[top_columns[x]], prefix=top_columns[x], prefix_sep=' ')
    onehot_df.insert(0, 'SalePrice', df[top_columns[0]])
    corr = onehot_df.corr().abs()['SalePrice']
    
    # Removes the duplicate SalePrice from the dataset
    corr = corr.where((corr.index != 'SalePrice'))
    corr.dropna(inplace=True)
    onehot_df = onehot_df[corr.index.tolist()]

    # Stores the data from the columns
    temp_df = onehot_df.merge(split_columns_df, how='outer', left_index=True, right_index=True)
    
# Adds in the SalePrice column and creates a correlation matrix
split_columns_df['SalePrice'] = df['SalePrice']
onehot_df = split_columns_df.corr('pearson').abs()

# Captures the 16 highest correlated columns, then stores the names of those columns as a list
onehot_df = onehot_df.sort_values('SalePrice', ascending=False)
top_onehot_columns = onehot_df.head(16)
top_onehot_columns = top_onehot_columns.index.tolist()

# Creates the heatmap to display the correlation
fig = px.imshow(onehot_df[top_onehot_columns].corr('pearson'), text_auto=True, aspect="auto")
fig.show()

This correlation matrix gives us a more precise view of the key features in our data. Some continuous features such as 'Year Built' and '1st Flr SF' maintain their higher correlation, but by creating one-hots from the categorical columns, you can see that certain categories are more important that others. For instance, 'Garage Cars 3.0' has a much stronger correlation to the price than 'Garage Cars 1.0', allowing us isolate the most important parts of given features. 

Before we can preform predictive analysis via machine learning, we need to clean dependent variable, SalePrice. SalePrice is still a continuous column which is very difficult to predict accurately. One method of alleviating this problem is to convert this to categorical values by using bins, or binning. To do so, we will define a range of bins, then place each observation into the appropriate bin.

In [191]:
import numpy as np

# Create a list of bins, ranging from the lowest to highest sales price, every $5,000
bins = list(range(split_columns_df['SalePrice'].min(), split_columns_df['SalePrice'].max(), 5000))

cleaned_df = split_columns_df.copy()
cleaned_df['SalePrice'] = np.searchsorted(bins, cleaned_df['SalePrice'].values)

Now that we have isolated the top columns in the dataset for correlation, and we have binned the SalePrice to be within $5000 of the original price, we can select a machine learning model to use as our classification model. With both our independent and dependant variables being categorical, the best option for classification would be a decision tree.

In [198]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

y = cleaned_df[top_onehot_columns].iloc[:, :1]
X = cleaned_df[top_onehot_columns].iloc[:, 1:]


X_train, X_test, y_train, y_test = train_test_split(cleaned_df[top_onehot_columns].apply(preprocessing.LabelEncoder().fit_transform), y, test_size=0.8, random_state=1)

TreeModel = DecisionTreeClassifier(random_state=1)
TreeModel = TreeModel.fit(X_train, y_train.values.ravel())
y_pred = TreeModel.predict(X_test)

#Prints the results of the decision tree
TreeAccuracy = accuracy_score(y_test, y_pred)
print(f"Decision Tree Accuracy: {TreeAccuracy}")

Decision Tree Accuracy: 0.9377133105802048
